In [2]:
import pandas as pd
import re
import itertools
import time

In [3]:
movies = pd.read_csv('../Data/movie_bd_v5.csv')
movies.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
790,tt1210166,50000000,110206216,Moneyball,Brad Pitt|Jonah Hill|Robin Wright|Philip Seymo...,Bennett Miller,What are you really worth?,The story of Oakland Athletics general manager...,133,Drama,Columbia Pictures|Scott Rudin Productions,9/22/2011,6.9,2011
142,tt1872181,200000000,705717432,The Amazing Spider-Man 2,Andrew Garfield|Emma Stone|Jamie Foxx|Dane DeH...,Marc Webb,No more secrets.,"For Peter Parker, life is busy. Between taking...",142,Action|Adventure|Fantasy,Columbia Pictures|Marvel Enterprises|Avi Arad ...,4/16/2014,6.6,2014
1677,tt0884328,18000000,57096190,The Mist,Thomas Jane|Marcia Gay Harden|Laurie Holden|To...,Frank Darabont,"Belief divides them, mystery surrounds them, b...","After a violent storm, a dense cloud of mist e...",126,Science Fiction|Horror|Thriller,Darkwoods Productions|Dimension Films,11/21/2007,6.6,2007
1615,tt0368933,40000000,95149435,The Princess Diaries 2: Royal Engagement,Anne Hathaway|Julie Andrews|HÃ©ctor Elizondo|J...,Garry Marshall,It can take a lifetime to find true love; she'...,Mia Thermopolis is now a college graduate and ...,113,Comedy|Drama|Family|Romance,Walt Disney Pictures,8/6/2004,5.9,2004
1406,tt0365485,10000000,17297244,The Matador,Pierce Brosnan|Greg Kinnear|Hope Davis|Portia ...,Richard Shepard,A hitman and a salesman walk into a bar...,"The life of Danny Wright, a salesman forever o...",96,Action|Comedy|Crime|Drama|Thriller,Miramax Films,5/12/2005,6.0,2005


In [4]:
movies.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [5]:
answers = {} # создадим словарь для ответов

# Prepare extended movies db for further using
# add profit column
movies_ext = movies.copy()
movies_ext['profit'] = movies_ext.revenue - movies_ext.budget

# and move it after 'revenue' (before 'original_title') column
cols = movies_ext.columns.tolist()
cols.insert(cols.index('original_title'), cols.pop())
movies_ext = movies_ext[cols]
print(f'Preprocessing: \'profit\' column was successfully added to movies db.')

# === Functions ===
def separated_text_to_list(s, sep='|'):
    return s.split(sep)

def extract_month(date_str):
    return int(separated_text_to_list(date_str, sep='/')[0])

def count_words(text):
    return len(re.findall(r'\w+', text))

Preprocessing: 'profit' column was successfully added to movies db.


# 1. У какого фильма из списка самый большой бюджет?

In [6]:
# +
answers['1'] = 'Pirates of the Caribbean: On Stranger Tides'

In [7]:
movies[movies.budget == movies.budget.max()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
723,tt1298650,380000000,1021683000,Pirates of the Caribbean: On Stranger Tides,Johnny Depp|PenÃ©lope Cruz|Geoffrey Rush|Ian M...,Rob Marshall,Live Forever Or Die Trying.,Captain Jack Sparrow crosses paths with a woma...,136,Adventure|Action|Fantasy,Walt Disney Pictures|Jerry Bruckheimer Films|M...,5/11/2011,6.3,2011


# 2. Какой из фильмов самый длительный (в минутах)?

In [8]:
# +
answers['2'] = 'Gods and Generals'

In [9]:
movies[movies.runtime == movies.runtime.max()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1157,tt0279111,56000000,12923936,Gods and Generals,Stephen Lang|Jeff Daniels|Robert Duvall|Kevin ...,Ronald F. Maxwell,The nations heart was touched by...,The film centers mostly around the personal an...,214,Drama|History|War,Turner Pictures|Antietam Filmworks,2/21/2003,5.8,2003


# 3. Какой из фильмов самый короткий (в минутах)?

In [10]:
# +
answers['3'] = 'Winnie the Pooh'

In [11]:
movies[movies.runtime == movies.runtime.min()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
768,tt1449283,30000000,14460000,Winnie the Pooh,Jim Cummings|Travis Oates|Jim Cummings|Bud Luc...,Stephen Anderson|Don Hall,Oh Pooh.,"During an ordinary day in Hundred Acre Wood, W...",63,Animation|Family,Walt Disney Pictures|Walt Disney Animation Stu...,4/13/2011,6.8,2011


# 4. Какова средняя длительность фильмов?

In [12]:
# +
answers['4'] = '110'

In [13]:
runtime_mean = movies.runtime.mean()
print(f'runtime_mean={runtime_mean} ({round(runtime_mean)})')

runtime_mean=109.6585494970884 (110)


# 5. Каково медианное значение длительности фильмов?

In [14]:
# +
answers['5'] = '107'

In [15]:
movies.runtime.median()

107.0

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [16]:
# +
answers['6'] = 'Avatar'

In [17]:
movies_ext[movies_ext.profit == movies_ext.profit.max()]

,imdb_id,budget,revenue,profit,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
239,tt0499549,237000000,2781505847,2544505847,Avatar,Sam Worthington|Zoe Saldana|Sigourney Weaver|S...,James Cameron,Enter the World of Pandora.,"In the 22nd century, a paraplegic Marine is di...",162,Action|Adventure|Fantasy|Science Fiction,Ingenious Film Partners|Twentieth Century Fox ...,12/10/2009,7.1,2009


# 7. Какой фильм самый убыточный? 

In [18]:
# +
answers['7'] = 'The Lone Ranger'

In [19]:
movies_ext[movies_ext.profit == movies_ext.profit.min()]

,imdb_id,budget,revenue,profit,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1245,tt1210819,255000000,89289910,-165710090,The Lone Ranger,Johnny Depp|Armie Hammer|William Fichtner|Hele...,Gore Verbinski,Never Take Off the Mask,The Texas Rangers chase down a gang of outlaws...,149,Action|Adventure|Western,Walt Disney Pictures|Jerry Bruckheimer Films|I...,7/3/2013,6.0,2013


# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [20]:
# +
answers['8'] = '1478'

In [21]:
len(movies[movies.revenue > movies.budget].index)

1478

ВАРИАНТ 2

In [22]:
len(movies_ext[movies_ext.profit > 0].index)

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [23]:
# +
answers['9'] = 'The Dark Knight'

In [24]:
movies_2008 = movies[movies.release_year == 2008]
movies_2008.iloc[movies_2008['revenue'].argmax()]

imdb_id                                                         tt0468569
budget                                                          185000000
revenue                                                        1001921825
original_title                                            The Dark Knight
cast                    Christian Bale|Michael Caine|Heath Ledger|Aaro...
director                                                Christopher Nolan
tagline                                                   Why So Serious?
overview                Batman raises the stakes in his war on crime. ...
runtime                                                               152
genres                                        Drama|Action|Crime|Thriller
production_companies    DC Comics|Legendary Pictures|Warner Bros.|Syncopy
release_date                                                    7/16/2008
vote_average                                                          8.1
release_year                          

ВАРИАНТ 2

In [25]:
movies[movies.release_year == 2008].sort_values(by='revenue', ascending=False)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
599,tt0468569,185000000,1001921825,The Dark Knight,Christian Bale|Michael Caine|Heath Ledger|Aaro...,Christopher Nolan,Why So Serious?,Batman raises the stakes in his war on crime. ...,152,Drama|Action|Crime|Thriller,DC Comics|Legendary Pictures|Warner Bros.|Syncopy,7/16/2008,8.1,2008
603,tt0367882,185000000,786636033,Indiana Jones and the Kingdom of the Crystal S...,Harrison Ford|Cate Blanchett|Shia LaBeouf|Ray ...,Steven Spielberg,The adventure continues . . .,"Set during the Cold War, the Soviets â€“ led b...",122,Adventure|Action,Lucasfilm|Paramount Pictures,5/21/2008,5.6,2008
606,tt0441773,130000000,631744560,Kung Fu Panda,Jack Black|Dustin Hoffman|Angelina Jolie|Jacki...,Mark Osborne|John Stevenson,Prepare for awesomeness.,"When the Valley of Peace is threatened, lazy P...",90,Animation|Family,DreamWorks Animation,6/4/2008,6.8,2008
621,tt0448157,150000000,624029371,Hancock,Will Smith|Charlize Theron|Jason Bateman|Jae H...,Peter Berg,Bad Behavior. Bad Attitude. Real Hero.,Hancock is a down-and-out superhero who's forc...,92,Fantasy|Action,Columbia Pictures|Weed Road Pictures|Forward P...,7/1/2008,6.0,2008
607,tt0795421,52000000,609841637,Mamma Mia!,Meryl Streep|Pierce Brosnan|Amanda Seyfried|Co...,Phyllida Lloyd,Take a trip down the aisle you'll never forget,"Set on an idyllic Greek island, the plot serve...",108,Comedy|Romance,Universal Pictures|Internationale Filmprodukti...,6/30/2008,6.2,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
708,tt0486674,25000000,6673422,What Just Happened,Bruce Willis|Robert De Niro|Sean Penn|Robin Wr...,Barry Levinson,Admit nothing.,During the course of an ordinary week in Holly...,104,Comedy|Drama,2929 Productions|Linson Entertainment|Art Lins...,10/17/2008,5.1,2008
673,tt0800241,15000000,5926410,Transsiberian,Woody Harrelson|Emily Mortimer|Kate Mara|Eduar...,Brad Anderson,You can't escape your lies.,A Trans-Siberian train journey from China to M...,111,Thriller|Crime|Mystery,Lietuvos Kinostudija|Scout Productions|Telecin...,1/18/2008,6.3,2008
702,tt0844286,20000000,5530764,The Brothers Bloom,Rachel Weisz|Adrien Brody|Mark Ruffalo|Rinko K...,Rian Johnson,They'd never let the truth come between them.,The Brothers Bloom are the best con men in the...,114,Adventure|Comedy|Drama|Romance,The Weinstein Company|Summit Entertainment|End...,9/9/2008,6.8,2008
711,tt0805570,15000000,3533227,The Midnight Meat Train,Bradley Cooper|Vinnie Jones|Brooke Shields|Les...,RyÃ»hei Kitamura,The most terrifying ride you'll ever take,The photographer Leon lives with his girlfrien...,98,Mystery|Drama|Crime|Thriller|Horror,Lions Gate Films|Lakeshore Entertainment|Green...,8/7/2008,5.7,2008


# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?

In [26]:
# +
answers['10'] = 'The Lone Ranger'

In [27]:
movies_2012_2014 = movies_ext[(movies_ext.release_year >= 2012) & (movies_ext.release_year <= 2014)]
movies_2012_2014.iloc[movies_2012_2014['profit'].argmin()]

imdb_id                                                         tt1210819
budget                                                          255000000
revenue                                                          89289910
profit                                                         -165710090
original_title                                            The Lone Ranger
cast                    Johnny Depp|Armie Hammer|William Fichtner|Hele...
director                                                   Gore Verbinski
tagline                                           Never Take Off the Mask
overview                The Texas Rangers chase down a gang of outlaws...
runtime                                                               149
genres                                           Action|Adventure|Western
production_companies    Walt Disney Pictures|Jerry Bruckheimer Films|I...
release_date                                                     7/3/2013
vote_average                          

ВАРИАНТ 2

In [28]:
movies_ext[(movies_ext.release_year >= 2012) & (movies_ext.release_year <= 2014)].sort_values(by='profit')

,imdb_id,budget,revenue,profit,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1245,tt1210819,255000000,89289910,-165710090,The Lone Ranger,Johnny Depp|Armie Hammer|William Fichtner|Hele...,Gore Verbinski,Never Take Off the Mask,The Texas Rangers chase down a gang of outlaws...,149,Action|Adventure|Western,Walt Disney Pictures|Jerry Bruckheimer Films|I...,7/3/2013,6.0,2013
1214,tt0790736,130000000,61648500,-68351500,R.I.P.D.,Jeff Bridges|Ryan Reynolds|Kevin Bacon|Stephan...,Robert Schwentke,To protect and serve the living,A recently slain cop joins a team of undead po...,96,Fantasy|Action|Comedy|Crime,Universal Pictures|Original Film|Dark Horse En...,7/18/2013,5.3,2013
1007,tt1374992,60000000,8106475,-51893525,Upside Down,Kirsten Dunst|Jim Sturgess|Timothy Spall|James...,Juan Diego Solanas,Two worlds. One future.,In an alternate universe where twinned worlds ...,104,Romance|Science Fiction|Drama|Fantasy,Onyx Films|Studio 37|Jouror Productions|France...,9/27/2012,6.1,2012
1302,tt0884726,70000000,18662027,-51337973,Legends of Oz: Dorothy's Return,Lea Michele|Dan Aykroyd|Patrick Stewart|Hugh D...,Dan St. Pierre,There's trouble in OZ...,"Dorothy wakes up in post-tornado Kansas, only ...",88,Animation|Music|Family,Prana Animation Studios|Summertime Entertainment,6/13/2013,6.1,2013
1235,tt1308729,55000000,9489829,-45510171,Bullet to the Head,Sylvester Stallone|Sung Kang|Sarah Shahi|Adewa...,Walter Hill,Revenge Never Gets Old.,"After watching their respective partners die, ...",92,Action|Crime|Thriller,Dark Castle Entertainment|After Dark Films|IM ...,1/31/2013,5.2,2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1189,tt1690953,76000000,970761885,894761885,Despicable Me 2,Steve Carell|Kristen Wiig|Benjamin Bratt|Miran...,Pierre Coffin|Chris Renaud,Back 2 Work,Gru is recruited by the Anti-Villain League to...,98,Animation|Comedy|Family,Universal Pictures|Illumination Entertainment,6/25/2013,7.0,2013
974,tt1074638,200000000,1108561013,908561013,Skyfall,Daniel Craig|Judi Dench|Javier Bardem|Ralph Fi...,Sam Mendes,Think on your sins.,When Bond's latest assignment goes gravely wro...,143,Action|Adventure|Thriller,Columbia Pictures,10/25/2012,6.8,2012
1180,tt1300854,200000000,1215439994,1015439994,Iron Man 3,Robert Downey Jr.|Gwyneth Paltrow|Guy Pearce|D...,Shane Black,Unleash the power behind the armor.,When Tony Stark's world is torn apart by a for...,130,Action|Adventure|Science Fiction,Marvel Studios,4/18/2013,6.9,2013
1177,tt2294629,150000000,1274219009,1124219009,Frozen,Kristen Bell|Idina Menzel|Jonathan Groff|Josh ...,Chris Buck|Jennifer Lee,Only the act of true love will thaw a frozen h...,Young princess Anna of Arendelle dreams about ...,102,Animation|Adventure|Family,Walt Disney Pictures|Walt Disney Animation Stu...,11/27/2013,7.5,2013


# 11. Какого жанра фильмов больше всего?

In [29]:
# +
answers['11'] = 'Drama'

In [30]:
genres_col = movies['genres'].apply(separated_text_to_list)
genres_col.explode().value_counts()

Drama              782
Comedy             683
Thriller           596
Action             582
Adventure          415
Crime              315
Romance            308
Family             260
Science Fiction    248
Fantasy            222
Horror             176
Mystery            168
Animation          139
Music               64
History             62
War                 58
Western             19
Documentary          8
Foreign              2
Name: genres, dtype: int64

# 12. Фильмы какого жанра чаще всего становятся прибыльными?

In [31]:
# +
answers['12'] = 'Drama'

In [32]:
movies_genres = movies_ext.copy()       # movies_ext contains 'profit' column
movies_genres['genres'] = movies_genres['genres'].apply(separated_text_to_list)
movies_genres[movies_genres.profit > 0].explode('genres')['genres'].value_counts()

Drama              560
Comedy             551
Thriller           446
Action             444
Adventure          337
Romance            242
Crime              231
Family             226
Science Fiction    195
Fantasy            188
Horror             150
Animation          120
Mystery            119
Music               47
History             46
War                 41
Western             12
Documentary          7
Name: genres, dtype: int64

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [33]:
# +
answers['13'] = 'Peter Jackson'

In [34]:
movies_director = movies.copy()
movies_director['director_revenue'] = movies_director.groupby('director')['revenue'].transform(sum)
movies_director.sort_values(by='director_revenue').tail(1)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,director_revenue
1081,tt0167260,94000000,1118888979,The Lord of the Rings: The Return of the King,Elijah Wood|Ian McKellen|Viggo Mortensen|Liv T...,Peter Jackson,The eye of the enemy is moving.,Aragorn is revealed as the heir to the ancient...,201,Adventure|Fantasy|Action,WingNut Films|New Line Cinema,12/1/2003,7.9,2003,6490593685


# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [35]:
# +
answers['14'] = 'Robert Rodriguez'

In [36]:
movies_director = movies_ext.copy()       # movies_ext contains 'profit' column
movies_director['director'] = movies_director['director'].apply(separated_text_to_list)
movies_director = movies_director.explode('director')
movies_director[movies_director.genres.str.contains('Action')]['director'].value_counts()

Robert Rodriguez        9
Michael Bay             7
Paul W.S. Anderson      7
Antoine Fuqua           6
Ridley Scott            6
                       ..
Luis Mandoki            1
Nicolas Winding Refn    1
Joel Coen               1
Tony Bill               1
Peter Farrelly          1
Name: director, Length: 364, dtype: int64

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году?

In [37]:
# +
answers['15'] = 'Chris Hemsworth'

In [38]:
movies_cast = movies.copy()
movies_cast = movies_cast[movies_cast.release_year == 2012]
movies_cast['cast'] = movies_cast['cast'].apply(separated_text_to_list)
movies_cast = movies_cast.explode('cast')
movies_cast[movies_cast.revenue == movies_cast.revenue.max()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
970,tt0848228,220000000,1519557910,The Avengers,Robert Downey Jr.,Joss Whedon,Some assembly required.,When an unexpected enemy emerges and threatens...,143,Science Fiction|Action|Adventure,Marvel Studios,4/25/2012,7.3,2012
970,tt0848228,220000000,1519557910,The Avengers,Chris Evans,Joss Whedon,Some assembly required.,When an unexpected enemy emerges and threatens...,143,Science Fiction|Action|Adventure,Marvel Studios,4/25/2012,7.3,2012
970,tt0848228,220000000,1519557910,The Avengers,Mark Ruffalo,Joss Whedon,Some assembly required.,When an unexpected enemy emerges and threatens...,143,Science Fiction|Action|Adventure,Marvel Studios,4/25/2012,7.3,2012
970,tt0848228,220000000,1519557910,The Avengers,Chris Hemsworth,Joss Whedon,Some assembly required.,When an unexpected enemy emerges and threatens...,143,Science Fiction|Action|Adventure,Marvel Studios,4/25/2012,7.3,2012
970,tt0848228,220000000,1519557910,The Avengers,Scarlett Johansson,Joss Whedon,Some assembly required.,When an unexpected enemy emerges and threatens...,143,Science Fiction|Action|Adventure,Marvel Studios,4/25/2012,7.3,2012


# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [39]:
# +
answers['16'] = 'Matt Damon'

In [40]:
movies_cast = movies.copy()
movies_cast['cast'] = movies_cast['cast'].apply(separated_text_to_list)
movies_cast[movies_cast.budget > movies_cast.budget.mean()].explode('cast')['cast'].value_counts()

Matt Damon               18
Adam Sandler             17
Angelina Jolie           16
Samuel L. Jackson        15
Tom Cruise               15
                         ..
Jason Biggs               1
Svetlana Khodchenkova     1
James Karen               1
Craig Ferguson            1
Ellen Page                1
Name: cast, Length: 1505, dtype: int64

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [41]:
# +
answers['17'] = 'Action'

In [42]:
movies_genres = movies_ext.copy()       # movies_ext contains 'profit' column
movies_genres['genres'] = movies_genres['genres'].apply(separated_text_to_list)
movies_genres = movies_genres.explode('genres')
movies_genres[movies_genres.cast.str.contains('Nicolas Cage')]['genres'].value_counts()

Action             17
Thriller           15
Drama              12
Crime              10
Fantasy             8
Adventure           7
Comedy              6
Science Fiction     4
Mystery             3
Family              3
Animation           3
History             2
War                 1
Horror              1
Romance             1
Name: genres, dtype: int64

# 18. Самый убыточный фильм от Paramount Pictures

In [43]:
# +
answers['18'] = 'K-19: The Widowmaker'

In [44]:
movies_prod_comp = movies_ext[movies_ext['production_companies'].str.contains('Paramount Pictures')]
movies_prod_comp[movies_prod_comp.profit == movies_prod_comp.profit.min()]

,imdb_id,budget,revenue,profit,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
925,tt0267626,100000000,35168966,-64831034,K-19: The Widowmaker,Harrison Ford|Liam Neeson|Peter Sarsgaard|Joss...,Kathryn Bigelow,Fate has found its hero.,When Russia's first nuclear submarine malfunct...,138,Thriller|Drama|History,Paramount Pictures|Intermedia Films|National G...,7/19/2002,6.0,2002


# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [45]:
# +
answers['19'] = '2015'

In [46]:
grouped_df = movies.groupby('release_year').sum()
grouped_df.sort_values('revenue', ascending=False)

,budget,revenue,runtime,vote_average
release_year,,,,
2015,6780630004,25449202382,13355,755.2
2014,7008050000,23405862953,13444,774.5
2013,7970620000,23213799791,15039,846.9
2012,7002000000,23079001687,12145,682.1
2011,7946550531,22676791872,15355,868.5
2010,7540000000,21071204961,13592,762.1
2009,6838046652,20261791024,14187,800.2
2008,6588900000,18252781990,12979,731.1
2007,6596495000,18162406801,14305,809.3


# 20. Какой самый прибыльный год для студии Warner Bros?

In [47]:
# +
answers['20'] = '2014'

In [48]:
movies_prod_comp = movies_ext[movies_ext['production_companies'].str.contains('Warner Bros')]
grouped_df = movies_prod_comp.groupby('release_year').sum()
grouped_df.sort_values('profit', ascending=False)

,budget,revenue,profit,runtime,vote_average
release_year,,,,,
2014,947600000,3243064519,2295464519,1576,88.9
2007,982000000,3183675217,2201675217,1721,95.4
2008,916000000,3050595031,2134595031,1366,72.0
2010,1034000000,3008712985,1974712985,1421,79.6
2011,974000000,2845393682,1871393682,1248,68.3
2003,890000000,2745493377,1855493377,1085,56.7
2009,915500000,2737954136,1822454136,1305,67.9
2013,963500000,2599953400,1636453400,1100,59.9
2004,1100000000,2731933725,1631933725,1624,78.7


# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [49]:
# +
answers['21'] = 'Сентябрь'

In [50]:
import calendar

movies_month = movies.copy()
movies_month['release_month'] = movies_month['release_date'].apply(extract_month)
month_idx = movies_month['release_month'].value_counts().index[0]
calendar.month_name[month_idx]

'September'

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [51]:
# +
answers['22'] = '450'

In [52]:
summer_months = [6, 7, 8]

movies_month = movies.copy()
movies_month['release_month'] = movies_month['release_date'].apply(extract_month)
len(movies_month[movies_month.release_month.isin(summer_months)].index)

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [53]:
# +
answers['23'] = 'Peter Jackson'

In [54]:
winter_months = [12, 1, 2]

# get only "winter movies"
movies_new = movies.copy()
movies_new['release_month'] = movies_new['release_date'].apply(extract_month)
movies_new = movies_new[movies_month.release_month.isin(winter_months)]

# solve for director
movies_new['director'] = movies_new['director'].apply(separated_text_to_list)
movies_new.explode('director')['director'].value_counts()

Peter Jackson        7
Clint Eastwood       6
Steven Soderbergh    6
Shawn Levy           4
Adam Shankman        4
                    ..
Sergey Bodrov        1
Eric Bress           1
Cory Edwards         1
Bill Condon          1
Pete Travis          1
Name: director, Length: 358, dtype: int64

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [55]:
# +
answers['24'] = 'Four By Two Productions'

In [56]:
movies_new = movies.copy()
movies_new['title_len'] = movies_new['original_title'].apply(lambda s: len(s))
movies_new['production_companies'] = movies_new['production_companies'].apply(separated_text_to_list)

movies_new = movies_new.explode('production_companies')
grouped_df = movies_new.groupby('production_companies').mean()
grouped_df[grouped_df.title_len == grouped_df.title_len.max()]

,budget,revenue,runtime,vote_average,release_year,title_len
production_companies,,,,,,
Four By Two Productions,18000000.0,261572744.0,82.0,6.4,2006.0,83.0


# 25. Описания фильмов какой студии в среднем самые длинные по количеству слов?

In [57]:
# +
answers['25'] = 'Midnight Picture Show'

In [58]:
movies_new = movies.copy()
movies_new['overview_len'] = movies_new['overview'].apply(count_words)
movies_new['production_companies'] = movies_new['production_companies'].apply(separated_text_to_list)

movies_new = movies_new.explode('production_companies')
grouped_df = movies_new.groupby('production_companies').mean()
grouped_df[grouped_df.overview_len == grouped_df.overview_len.max()]

,budget,revenue,runtime,vote_average,release_year,overview_len
production_companies,,,,,,
Midnight Picture Show,15000000.0,3533227.0,98.0,5.7,2008.0,176.0


# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [59]:
# +
answers['26'] = 'Inside Out, The Dark Knight, 12 Years a Slave верно'

In [60]:
movies[['original_title','vote_average']].sort_values(by='vote_average').tail(int(len(movies.index)/100))

,original_title,vote_average
155,The Theory of Everything,7.8
863,The Lord of the Rings: The Two Towers,7.8
1688,There Will Be Blood,7.8
283,Mr. Nobody,7.8
1191,12 Years a Slave,7.9
872,The Pianist,7.9
128,Gone Girl,7.9
1800,Memento,7.9
119,Guardians of the Galaxy,7.9
370,Inception,7.9


ВАРИАНТ 2

In [61]:
# NB! This 2nd solution via quantile works a bit tricky on this db.
# There are more (for >=) or less (for >) rows selected than exactly 10% of movies
movies[['original_title','vote_average']][movies.vote_average > movies.vote_average.quantile(0.99)]

,original_title,vote_average
9,Inside Out,8.0
34,Room,8.0
118,Interstellar,8.0
119,Guardians of the Galaxy,7.9
125,The Imitation Game,8.0
128,Gone Girl,7.9
138,The Grand Budapest Hotel,7.9
370,Inception,7.9
599,The Dark Knight,8.1
872,The Pianist,7.9


# 27. Какие актеры чаще всего снимаются в одном фильме вместе?

In [62]:
# +
answers['27'] = 'Daniel Radcliffe, Rupert Grint'

In [63]:
cast_column = movies['cast'].apply(separated_text_to_list)

total_rows = len(movies.index)
proceed_actors = []
top_pair = None
num_common_movies_max = 0

start = time.time()

for index, cast in cast_column.iteritems():
    for actor_pair in itertools.combinations(cast, 2):
        actor_pair_set = set(actor_pair)

        # if current pair was not proceed yet
        if actor_pair_set not in proceed_actors:
            # common_movies = movies[movies.cast.str.contains(fr'(?=.*{actor_pair[0]})(?=.*{actor_pair[1]})', regex=True)]
            common_movies = movies[movies.cast.str.contains(actor_pair[0])
                                 & movies.cast.str.contains(actor_pair[1])]
            num_common_movies_cur = len(common_movies.index)

            if num_common_movies_cur > num_common_movies_max:
                num_common_movies_max = num_common_movies_cur
                top_pair = actor_pair

            # add actor pair to already proceed actors
            proceed_actors.append(actor_pair_set)

    # just out some progress
    print(f'{index/(total_rows-1):.2%} done', end="\r", flush=True)

end = time.time()
print()
print(f'top_pair={top_pair} num_common_movies_max={num_common_movies_max}, time={end-start}')

100.00% done
top_pair=('Daniel Radcliffe', 'Rupert Grint') num_common_movies_max=8, time=40.15754771232605


In [64]:
movies.cast[0]
# movies.cast.str.contains(fr'(?=.*{actor_pair[0]})(?=.*{actor_pair[1]})|', regex=True)

"Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vincent D'Onofrio|Nick Robinson"

In [65]:
movies.cast.str.contains(f'(?=.*Chris)(?=.*Dallas)', regex=True)

0        True
1       False
2       False
3       False
4       False
        ...  
1884    False
1885    False
1886    False
1887    False
1888    False
Name: cast, Length: 1889, dtype: bool

ВАРИАНТ 2

In [66]:
# NB! Much more slower via one invocation with regex in str.contains..

cast_column = movies['cast'].apply(separated_text_to_list)

total_rows = len(movies.index)
proceed_actors = []
top_pair = None
num_common_movies_max = 0

start = time.time()

for index, cast in cast_column.iteritems():
    for actor_pair in itertools.combinations(cast, 2):
        actor_pair_set = set(actor_pair)

        # if current pair was not proceed yet
        if actor_pair_set not in proceed_actors:
            common_movies = movies[movies.cast.str.contains(fr'(?=.*{actor_pair[0]})(?=.*{actor_pair[1]})', regex=True)]
            num_common_movies_cur = len(common_movies.index)

            if num_common_movies_cur > num_common_movies_max:
                num_common_movies_max = num_common_movies_cur
                top_pair = actor_pair

            # add actor pair to already proceed actors
            proceed_actors.append(actor_pair_set)

    # just out some progress
    print(f'{index/(total_rows-1):.2%} done', end="\r", flush=True)

end = time.time()
print()
print(f'top_pair={top_pair} num_common_movies_max={num_common_movies_max}, time={end-start}')

100.00% done
top_pair=('Daniel Radcliffe', 'Rupert Grint') num_common_movies_max=8, time=228.58824396133423


# Submission

In [67]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides',
 '2': 'Gods and Generals',
 '3': 'Winnie the Pooh',
 '4': '110',
 '5': '107',
 '6': 'Avatar',
 '7': 'The Lone Ranger',
 '8': '1478',
 '9': 'The Dark Knight',
 '10': 'The Lone Ranger',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker',
 '19': '2015',
 '20': '2014',
 '21': 'Сентябрь',
 '22': '450',
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': 'Inside Out, The Dark Knight, 12 Years a Slave верно',
 '27': 'Daniel Radcliffe, Rupert Grint'}

In [68]:
# и убедиться что ни чего не пропустил)
len(answers)

27